<a href="https://colab.research.google.com/github/hatemamine/Improving-Arabic-IR-and-Reranking-Performance-with-Knowledge-Distillation/blob/main/arabic_mmarco_firststage_faiss_kd_vs_nokd_bi_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/facebookresearch/faiss/wiki/Running-on-GPUs

In [1]:
!pip install faiss-cpu -q
!pip install huggingface_hub -q
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; plat

In [2]:
# embedding vector generated with our bi-encoder model
organization_dataset_id="hatemestinbejaia/ExperimentDATA_knowledge_distillation_vs_fine_tuning"
mmarco_embedding_collection_list =["collection-arabic-embedding-KDaradpr", "collection-arabic-embedding-KDaraelectra", "collection-arabic-embedding-KDmMinilm", "collection-arabic-embedding-NOKDaradpr", "collection-arabic-embedding-NOKDaraelectra", "collection-arabic-embedding-NOKDmMinilm"]
mmarco_embedding_queries_list=["queries-arabic-embedding-kdaradpr", "queries-arabic-embedding-kdaraelectra", "queries-arabic-embedding-kdmMinilm", "queries-arabic-embedding-NOKDaradpr", " queries-arabic-embedding-NOKDaraelectra", " queries-arabic-embedding-NOKDmMinilm"]

In [3]:
import numpy as np
import gc
from datasets import load_dataset
output_run="mmarco_first_stage_"
def combine_embedding(embedding1, embedding2):
    global output_run
    output_run+= mmarco_embedding_collection_list[embedding1]+mmarco_embedding_collection_list[embedding2]
    # load corpus embedding
    corpus_embeddings0 = load_dataset(organization_dataset_id, mmarco_embedding_collection_list[embedding1])
    print(corpus_embeddings0)
    corpus_embeddings1 = load_dataset(organization_dataset_id, mmarco_embedding_collection_list[embedding2])
    print(corpus_embeddings1)
    corpus_embeddings0=corpus_embeddings0["collection"].with_format("np")
    corpus_id=corpus_embeddings0["id"]
    corpus_embeddings0=corpus_embeddings0["embedding"]
    corpus_embeddings0
    # concatenate  embedding
    corpus_embeddings1=corpus_embeddings1["collection"].with_format("np")
    corpus_embeddings1=corpus_embeddings1["embedding"]
    corpus_embeddings1
    corpus_embeddings =np.concatenate((corpus_embeddings0,corpus_embeddings1),1)
    del corpus_embeddings0
    del corpus_embeddings1
    # Force a garbage collection
    gc.collect()
    # load queries embedding
    queries_embeddings0 = load_dataset(organization_dataset_id, mmarco_embedding_queries_list[embedding1], trust_remote_code=True)
    queries_embeddings1 = load_dataset(organization_dataset_id, mmarco_embedding_queries_list[embedding2], trust_remote_code=True)
    queries_embeddings0 = queries_embeddings0["dev"].with_format("np")
    queries_embeddings1 = queries_embeddings1["dev"].with_format("np")
    queries_id=queries_embeddings0["id"]
    queries_embeddings0=queries_embeddings0["embedding"]
    queries_embeddings1=queries_embeddings1["embedding"]
    queries_embeddings =np.concatenate((queries_embeddings0,queries_embeddings1),1)
    #print(queries_embeddings)
    print(len(queries_embeddings[0]))
    return corpus_embeddings,corpus_id, queries_embeddings, queries_id

def load_embedding(embedding):
    global output_run
    output_run+= mmarco_embedding_collection_list[embedding]
    # load corpus embedding
    corpus_embeddings = load_dataset(organization_dataset_id, mmarco_embedding_collection_list[embedding])
    print(corpus_embeddings)
    corpus_embeddings=corpus_embeddings["collection"].with_format("np")
    corpus_id=corpus_embeddings["id"]
    corpus_embeddings=corpus_embeddings["embedding"]
    corpus_embeddings
    # Force a garbage collection
    gc.collect()
    # load queries embedding
    queries_embeddings = load_dataset(organization_dataset_id, mmarco_embedding_queries_list[embedding], trust_remote_code=True)
    print(queries_embeddings)
    queries_embeddings = queries_embeddings["dev"].with_format("np")
    queries_id=queries_embeddings["id"]
    queries_embeddings=queries_embeddings["embedding"]
    #print(queries_embeddings)
    print(len(queries_embeddings[0]))
    return corpus_embeddings,corpus_id, queries_embeddings, queries_id

In [ ]:
# you can combine embedding vector from diffrent bi-encoder model
# Some models need more RAM than it's available in free Kaggle; you can go with mMiniLM
combine=False
if combine:corpus_embeddings,corpus_id, queries_embeddings, queries_id=combine_embedding(0, 2)
else: corpus_embeddings,corpus_id, queries_embeddings, queries_id=load_embedding(2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/27.2k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

collection-00000-of-00028.parquet:   0%|          | 0.00/684M [00:00<?, ?B/s]

In [ ]:
"""
This example uses Approximate Nearest Neighbor Search (ANN) with FAISS (https://github.com/facebookresearch/faiss).
"""

import csv
import os
import pickle
import time

import faiss
import numpy as np

embedding_size = len(corpus_embeddings[0])  # Size of embeddings
# Defining our FAISS index
# Number of clusters used for faiss. Select a value 4*sqrt(N) to 16*sqrt(N) - https://github.com/facebookresearch/faiss/wiki/Guidelines-to-choose-an-index
#n_clusters = 10
# We use Inner Product (dot-product) as Index. We will normalize our vectors to unit length, then is Inner Product equal to cosine similarity
#quantizer = faiss.IndexFlatIP(embedding_size)
#print("quantizer")
#index = faiss.IndexIVFFlat(quantizer, embedding_size, n_clusters, faiss.METRIC_INNER_PRODUCT)
#print("index")
# Number of clusters to explorer at search time. We will search for nearest neighbors in 3 clusters.
#index.nprobe = 10

### Create the FAISS index
print("Start creating FAISS index")
#index = faiss.IndexIDMap(faiss.IndexFlatL2(embedding_size))
#index = faiss.IndexFlatL2(embedding_size)
index = faiss.index_factory(embedding_size, "Flat", faiss.METRIC_INNER_PRODUCT)
#corpus_embeddings = corpus_embeddings / np.linalg.norm(corpus_embeddings, axis=1)[:, None]
faiss.normalize_L2(corpus_embeddings)
# First, we need to normalize vectors to unit length
#corpus_embeddings = corpus_embeddings / np.linalg.norm(corpus_embeddings, axis=1)[:, None]
print("norm")
# Then we train the index to find a suitable clustering
index.train(corpus_embeddings)
print("train")

In [ ]:
faiss.write_index(index,"mmarco.index")
index = faiss.read_index("mmarco.index")
index.add(corpus_embeddings)
faiss.write_index(index, "mmarco.index")
print("faiss index built it ok")
print("total index:",index.ntotal)

In [ ]:
# free some RAM
del corpus_embeddings
# Force a garbage collection
gc.collect()
#print(queries_embeddings[0])
top_k_hits = 1000
output_run += "_top_k_hits_"+str(top_k_hits) +".tsv"
print(output_run)

In [ ]:
corpus_id

In [ ]:
######### Search in the index ###########
import time
import numpy as np
from tqdm import trange
with open(output_run, 'w', encoding='utf-8') as f_out:
    for i in trange(len(queries_embeddings)):
        start_time = time.time()
        question_embedding = queries_embeddings[i]
        # FAISS works with inner product (dot product). When we normalize vectors to unit length, inner product is equal to cosine similarity
        #question_embedding = question_embedding / np.linalg.norm(question_embedding)
        question_embedding = np.expand_dims(question_embedding, axis=0)
        faiss.normalize_L2(question_embedding)

        # Search in FAISS. It returns a matrix with distances and corpus ids.
        distances, corpus_ids = index.search(question_embedding, top_k_hits)

        # We extract corpus ids and scores for the first query
        hits = [{"corpus_id": id, "score": score} for id, score in zip(corpus_ids[0], distances[0])]
        #print(hits)
        hits = sorted(hits, key=lambda x: x["score"], reverse=True)
        end_time = time.time()
        for j in range(len(hits)):
            f_out.write(f'{queries_id[i]}\t{corpus_id[hits[j]["corpus_id"]]}\t{j+1}\n')
        break

In [ ]:
print(end_time-start_time)

In [ ]:
!pip install -U datasets huggingface-hub -q
import huggingface_hub
hf = huggingface_hub.HfFolder()
# to load the reranked list to your Hugging Face repository, enter your access token below.
# and change the repository ID to yours.
#organization_dataset_id =""
access_token = "Hagging face access token"
hf.save_token(access_token)
from huggingface_hub import HfApi
api = HfApi()
#api.upload_file(path_or_fileobj=output_run, path_in_repo=output_run, repo_id=organization_dataset_id, repo_type="dataset",)